In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from imblearn.over_sampling import SMOTE

from fancyimpute import IterativeImputer

from scipy.stats import friedmanchisquare

import openml


In [2]:
diabetes_oml = openml.datasets.get_dataset(37)
X_diabetes, y_diabetes, _, _ = diabetes_oml.get_data(target=diabetes_oml.default_target_attribute)

cancer_oml = openml.datasets.get_dataset(15)
X_cancer, y_cancer, _, _ = cancer_oml.get_data(target=cancer_oml.default_target_attribute)

print("✅ Diabetes Dataset:")
print(X_diabetes.shape)
print(X_diabetes.head())

print("\n✅ Breast Cancer Dataset:")
print(X_cancer.shape)
print(X_cancer.head())

✅ Diabetes Dataset:
(768, 8)
   preg   plas  pres  skin   insu  mass   pedi   age
0   6.0  148.0  72.0  35.0    0.0  33.6  0.627  50.0
1   1.0   85.0  66.0  29.0    0.0  26.6  0.351  31.0
2   8.0  183.0  64.0   0.0    0.0  23.3  0.672  32.0
3   1.0   89.0  66.0  23.0   94.0  28.1  0.167  21.0
4   0.0  137.0  40.0  35.0  168.0  43.1  2.288  33.0

✅ Breast Cancer Dataset:
(699, 9)
   Clump_Thickness  Cell_Size_Uniformity  Cell_Shape_Uniformity  \
0              5.0                   1.0                    1.0   
1              5.0                   4.0                    4.0   
2              3.0                   1.0                    1.0   
3              6.0                   8.0                    8.0   
4              4.0                   1.0                    1.0   

   Marginal_Adhesion  Single_Epi_Cell_Size  Bare_Nuclei  Bland_Chromatin  \
0                1.0                   2.0          1.0              3.0   
1                5.0                   7.0         10.0        